<p style="text-align: center;" ><font size="7" >Creation of the PePrMInt Dataset</font></p>
<hr />

**Author**: Thibault Tubiana  
**Version**: 1.0  
**Last change**: Major refactoring

# **PREPARATION**

In [1]:
RECALCULATION = False 


## Depandancies
 --> See `00-SETYP.ipynb` for depandancies and configuration

 

## Preparation
 --> Please run `01-download_files.ipynb` before running this notebook, it countains all the code to download the required files

## Import packages

In [2]:
import pandas as pd
import numpy as np
import os
import importlib

#Pandas configuration
pd.options.mode.chained_assignment = (
    None  # default='warn', remove pandas warning when adding a new column
)

pd.set_option("display.max_columns", None)

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
%config InlineBackend.figure_format ='svg' #better quality figure figure

from tqdm.notebook import tnrange, tqdm
tqdm.pandas()  # activate tqdm progressbar for pandas apply
import ipywidgets as widgets
from IPython.display import display


/Users/thibault/miniconda3/envs/peprmint/lib/python3.7/site-packages/tqdm/std.py:699: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


## Import all global variables and setings

In [3]:
%run "./00-SETUP.ipynb"

- **Choose here** if a **full recomputation** is needed or if loading a checkpoint after the initial structural dataset computation is enough.  
  This is usefull if you want 

In [4]:
recalculation_widget = widgets.ToggleButton(
    value=RECALCULATION,
    description='Recalculation ?',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click for recalculation',
    icon='cogs' # (FontAwesome names without the `fa-` prefix)
)
display(recalculation_widget)

ToggleButton(value=False, description='Recalculation ?', icon='cogs', tooltip='Click for recalculation')

## Instanciating the builder object

In [5]:
# import builder.Builder as builderEngine
# importlib.reload(builderEngine)

# builder = builderEngine.Builder(SETUP, recalculate = recalculation_widget.value, update=False, notebook = True, core=1)

import pepr2ds.builder.Builder as builderEngine
importlib.reload(builderEngine)
builder = builderEngine.Builder(SETUP, recalculate = recalculation_widget.value, update=False, notebook = True, core=1)

<module 'pepr2ds.builder.Builder' from '/Users/thibault/OneDrive - University of Bergen/projects/peprmint/dev/pepermintdataset/pepr2ds/builder/Builder.py'>

# **STRUCTURAL DATASET**
## PDB Parsing and computations

The first step is to create a dataset based on all PDBs. Here's a quick description
1. For each CATH domains, all the PDBS (previously downloaded with `01-download_cathpdb.ipynb`) are **loaded one by one** and **cleaned**.
2. The PDBs will be **parsed with biopandas** to be transformed in a **DataFrame**. It will looks like this:  
    ![Example biopandas (need internet connectivity)](ressources/biopandas.png) 
3. The **secondary structure** will be calculated with [**DSSP**  ](https://swift.cmbi.umcn.nl/gv/dssp/)  
    ![secondary_structures](ressources/secondaryStructures.png)
4. The **Accessible Surface Area** is then computed with [**FREESASA**](http://freesasa.github.io/python/functions.html) (and DSSP..)  
    <div><img src="ressources/surface-diagram.png" width="400"></div>
5. **Proteins block** are calculated with [**PBxplore**](https://github.com/pierrepo/PBxplore) (check https://github.com/pierrepo/PBxplore)  
    <div><img src="ressources/PBs.jpg" width="400"></div>
6. Then everything will be merged and saved in the `WORKDIR/dataset`  
    ![secondary_structures](ressources/floppy.png)


### Cleaning

In [6]:
builder.structure.clean_all_pdbs()

In [7]:
DATASET = builder.structure.build_structural_dataset()

> Reading checkpoint 1


In [8]:
DATASET.data_type.unique()

array(['cathpdb', 'alfafold'], dtype=object)

## Computing protrusion models
Based on atom properties of all PDBs (which are in a dataset datastructure), we will compute **convexhull**, **vertices**, **protrusions**, **co-insertables** according Fuglebakk *et al.*, PONe, 2018: https://doi.org/10.1371/journal.pcbi.1006325  
![](ressources/protrusions.png)

In [9]:
DATASET = builder.structure.add_protrusions(DATASET)

... Conputing protrusions .. 


## Adding structural cluster information

To remove redundancy on our dataset (or filter based on redundancy later on), we need cluster informations. For that, we will use <img src="ressources/cathlogo.png" width="40"> clusters

In CATH, we can find info about the the sequence cluster.  
Note that there is 4 clusters:
 - S35 : Structure with 35% of sequence idendity
 - S60 : Structure with 65% of sequence idendity
 - S95 : Structure with 95% of sequence idendity
 - S100 : Structure with 100% of sequence idendity

But the cluster number is hierarchical actually.  
```
S35
 |_S60
    |_S95
       |_S100
```
So we need to make a transformation to keep the cluster info. 
For example, a sequence that is in the cluster 2 at 35%, cluster 4 at 60%, cluster 3 at 95% and cluster 1 at 100% will be `2.4.3.1`

In [11]:
import pepr2ds.builder.Builder as builderEngine
importlib.reload(builderEngine)
builder = builderEngine.Builder(SETUP, recalculate = recalculation_widget.value, update=False, notebook = True, core=1)

DATASET = builder.structure.add_structural_cluster_info(DATASET)

<module 'pepr2ds.builder.Builder' from '/Users/thibault/OneDrive - University of Bergen/projects/peprmint/dev/pepermintdataset/pepr2ds/builder/Builder.py'>

# **WORKING WITH SEQUENCES**  
Now that we are done with pdb files, with can work on sequences and other external databases


## Adding uniprot information (origin and uniprot_id)

From the PDBid, we will fetch **origin** and **uniprot_id** (like `ASAP1_HUMAN`) from the **uniprot_acc** (`Q9ULH1`)

In [12]:
DATASET = builder.sequence.add_uniprotId_Origin(DATASET)


## Starting to add sequences in DATASET

We can start to add sequences in our dataset from **PROSITE**. 
1. We take the multiple alignment from **prosite** and get all sequences.
2. for now we keep only sequences that are already in the DATASET (sequences that have a PDB structure)
3. we match the residue number of each residues in the PDB, with the residue position in the alignment
4. we add all those new data in the DATASET

In [13]:
DATASET = builder.sequence.match_residue_number_with_alignment_position(DATASET)


> Matching between the structure and the sequence aligned...


## Adding sequence without structures  

Of course all sequences doesn't have a structure, but they can be quite usefull for certain statistics (amino acid composition, conservation..), so we add them in our dataset even if they don't have a structure 🙂.  
For every amino acid of every sequences, we will consider the residue as a `CA` atom, to match the current dataframe structure

In [ ]:
DATASET = builder.sequence.add_sequence_in_dataset(DATASET)

PH
C2
C1
PX
FYVE
BAR
ENTH
SH2
SEC14
START
C2DIS
GLA
PLD
PLA
ANNEXIN

> Adding the Sequence data in the dataset...


## Adding info from uniprot protein sheet

Uniprot page can have a lot of usefull data (taxon, origin crossreferences). First we will download all UNIPROT xml sheet for every `uniprotid` we have in our dataset and then we parse it to get the data we want.  
For now the data we keep are 
 - [x] `uniprot_id` (for matching)
 - [x] `uniprot_acc` (for matching)
 - [x] `location` in the cell
 - [x] `taxon` for species 
 - [x] `CR:prositeID` Cross reference with PROSITE ID like `PS50003`
 - [x] `CR:prositeName` for prosite name (like `PH`)

In [ ]:
builder.sequence.download_uniprot_data(DATASET)
DATASET = builder.sequence.add_info_from_uniprot(DATASET)


> 0 new files downloaded and 6287 will be reused.
> Parsing uniprot files


## Adding Cluster info 

To remove the redundancy we can use the "Uniref" cluster info. There are 3 differents cluster level:
- `uniref50` at 50% of sequence idendity 
- `uniref90` at 90% of sequence idendity 
- `uniref100` at 100% of sequence idendity  

Every sequences with more than `50`/`90`/`100`% of sequence identity are cluseters together.  
More info can be found here https://www.uniprot.org/help/uniref

In [ ]:
DATASET = builder.sequence.add_cluster_info(DATASET)


trying to fetch NF50
  >done
trying to fetch NF90
  >done
trying to fetch NF100
  >done
> mapping with dataset
  >ok<  


## Adding Conservation

Per-position conservation is important to have information on the importance of each amino acid in the protein. If a residue is conserved it means that it can have a critical role for the protein.  
For now we will use only one formula to calculate the conservation: The **shannon entropy**.  
Shanon Entropy(Shannon, 1948) is possibly the most sensitive tool to estimate the diversity of a system (http://imed.med.ucm.es/Tools/svs_help.html#ref).  
For a multiple protein sequence alignment the Shannon entropy H for every position of the alignment is:  
$$H=-\sum_{i=1}^{M} P_i\,log_2\,P_i$$

 - Where $M$ is the number of different amino acids 
 - $P_i$ is the 'propability' to have a specific amino acid on a column (number of occurences of this amino acid on $M$)

From the github package by Felix Francis: https://github.com/ffrancis/Multiple-sequence-alignment-Shannon-s-entropy/blob/master/msa_shannon_entropy012915.py

- H ranges from 0 (only one base/residue in present at that position) to 4.322 (all 20 residues are equally represented in that position).  
- Typically, positions with H >2.0 are considerered variable, whereas those with H < 2 are consider conserved.  
- Highly conserved positions are those with H <1.0 (Litwin and Jores, 1992). A minimum number of sequences is however required (~100) for H to describe the diversity of a protein family.  

*Shannon, C. E. (1948) The mathematical theory of communication. The Bell system Technical Journal, 27, 379-423 & 623-656.*  
*Litwin S., Jores R. (1992) Shannon Information as a Measure of Amino Acid Diversity. In: Perelson A.S., Weisbuch G. (eds) Theoretical and Experimental Insights into Immunology. NATO ASI Series (Series H: Cell Biology), vol 66. Springer, Berlin, Heidelberg*

**We will use 2 Shannon entropy calculation** :
 - The regular one. All amino acids are considered are "unique". $Entropy_{M = 21} \in [0,4.33 \pm 0.01]$ estimated on 10000 repetitions of $N=10000$ random draw amoung 20 amino acids + gaps.
 - The H10 one. Amino acids are regroupes according their type (see http://thegrantlab.org/bio3d/reference/entropy.html), $Entropy_{M = 10}\in [0,3.1 \pm 0.01]$ estimated on 10000 repetitions of $N=10000$ random draw amoung 10 groups.
   `Hydrophobic/Aliphatic [V,I,L,M], Aromatic [F,W,Y], Ser/Thr [S,T], Polar [N,Q], Positive [H,K,R], Negative [D,E], Tiny [A,G], Proline [P], Cysteine [C], and Gaps [-,X].`  
    
**NOTE** : The conservation is normalized between 0 and 1, 0 is not conserved, 1 is fully conserved

In [ ]:
import importlib
importlib.reload(builderEngine)
builder = builderEngine.Builder(SETUP, recalculate = recalculation_widget.value, notebook = True)


DATASET = builder.sequence.add_conservation(DATASET, gapcutoff=0.8)

<module 'builder.Builder' from '/Users/thibault/OneDrive - University of Bergen/projects/peprmint/dev/pepermintdataset/notebooks/builder/Builder.py'>

INFO: Pandarallel will run on 4 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.
INFO: Pandarallel will run on 4 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.
INFO: Pandarallel will run on 4 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


# Saving dataset

## size optimisation
Last thing to do before saving, is optimizing the dataset size. This will improve computational time (and disk storage of course 🙂 )
One way to do it, it's to simplify the variables type. For example, change `str` to `category`" will help decrease the space in memory a lot for variables like `domain` or `resdidue_name`


In [ ]:
DATASET = builder.optimize_size(DATASET)


> datatypes optimisation
Size BEFORE optimization 4668.52 MB
Size AFTER optimization 2655.76 MB
56.89% of the original size


### Remove duplicated residues

I don't know why, but sometimes I have duplicated residues (the whole cathpdb is duplicated) so just in case, we remove the duplicates based on `'atom_number','atom_name','residue_name','residue_number','cathpdb','chain_id`

In [ ]:
DATASET = DATASET.drop_duplicates(subset=['atom_number','atom_name','residue_name','residue_number','cathpdb','chain_id'])

## Final saving

**FINALY!** We have our final dataset !!! 
We save it into 2 versions: Full (with all PDB atoms) and light (only with `CA` and `CB` atoms).  
➡️  For now we don't need other atoms for analysis, just CA and CB are enough and it can make the analysis faster.

In [ ]:
builder.save_checkpoint_dataset(DATASET,"DATASET_peprmint_allatoms_d25")
builder.save_checkpoint_dataset(DATASET.query("atom_name in ['CA','CB']"),"DATASET_peprmint_d25")

In [21]:
list(DATASET.domain.unique())

['PX',
 'PLD',
 'C2DIS',
 'SH2',
 'ENTH',
 'FYVE',
 'C1',
 'START',
 'ANNEXIN',
 'GLA',
 'PLA',
 'PH',
 'SEC14',
 'BAR',
 'C2']

In [19]:
#Generate alignment file list for C2DIS domain (S95, because everything is just too slow, too much structure)

def selectUniquePerCluster(df, cathCluster, Uniref, withAlignment=True, pdbreference=None,
                               removeStrand=False):
        """
        Return a datasert with only 1 data per choosed clusters.
        """

        if cathCluster not in ["S35", "S60", "S95", "S100"]:
            raise ValueError('CathCluster given not in ["S35","S60","S95","S100"]')

        if Uniref not in ["uniref50", "uniref90", "uniref100"]:
            raise ValueError('CathCluster given not in ["uniref50","uniref90","uniref100"]')

        if withAlignment:
            df = df[~df.alignment_position.isnull()]

        cathdf = df.query("data_type == 'cathpdb'")
        seqdf = df.query("data_type == 'prosite'")

        def selectUniqueCath(group):
            uniqueNames = group.cathpdb.unique()
            if pdbreference:
                if pdbreference in uniqueNames:
                    select = pdbreference
                else:
                    select = uniqueNames[0]
            else:
                select = uniqueNames[0]

            # return group.query("cathpdb == @select")
            return select

        def selectUniqueUniref(group, exclusion):
            uniqueNames = group.uniprot_acc.unique()
            select = uniqueNames[0]
            # return group.query("uniprot_acc == @select")
            if select not in exclusion:
                return select
        dfReprCathNames = cathdf.groupby(["domain", cathCluster]).apply(selectUniqueCath).to_numpy()
        print(dfReprCathNames)
        excludeUniref = df.query("cathpdb in @dfReprCathNames").uniprot_acc.unique()  # Structures are prior to sequences.
        dfReprUnirefNames = seqdf.groupby(["domain", Uniref]).apply(selectUniqueUniref,
                                                                    exclusion=excludeUniref).to_numpy()
        dfReprCath = cathdf.query("cathpdb in @dfReprCathNames")
        dfReprUniref = seqdf.query("uniprot_acc in @dfReprUnirefNames")

        return (pd.concat([dfReprCath, dfReprUniref]))

    
c2dis = selectUniquePerCluster(DATASET.query("domain== 'PH'"), 'S95', 'uniref90', withAlignment=False, pdbreference='2da0A00')
#pdblist = c2dis.cathpdb.dropna().unique()

['1unpA00' '2x18G00' '1eazA00' '2hthB00' '2dx5A00' '1rrpD00' '1shcA00'
 '3ml4A02' '2dyqA00' '3hk0A02' '4k81E02' '4k17B01' '4k17C01' '4k17A01'
 '4k17D01' '5efxA00' '3pvlA04' '1pmsA00' '2dtcB00' '3mpxA02' '2yf0A01'
 '2rgnB02' '3f0wA00' '2nmbA00' '1wj1A01' '4gzuB03' '1foeE02' '4gzuB02'
 '1v5uA00' '1wgqA00' '2d9yA00' '2yryA00' '2dkpA01' '2rovA00' '4tyzB00'
 '1v89A00' '2coaA01' '2d9zA01' '2ec1A00' '2m14A00' '2dn6A00' '2mfqA00'
 '1fhoA00' '1plsA00' '2ys1A00' '1wi1A01' '3f5rA00' '4khbD01' '1tqzA01'
 '2d9wA01' '1z87A01' '5gowB00' '2cofA00' '2rloA00' '1v88A00' '1v5pA01'
 '1v61A00' '2cocA01' '2iybD00' '1qc6A00' '1egxA00' '2dhkA01' '2m38A00'
 '1v5mA00' '2codA01' '1xr0B01' '2ys5A01' '1mkeA00' '1x1fA00' '2fjlA00'
 '1wg7A01' '2yt0A01' '2i5fA00' '1x05A00' '1x1gA00' '2p8vA00' '1ddvA00'
 '1i7aD00' '2qkmC00' '5dh9B00' '4l6eA00' '1k5dB00' '1xkeA00' '1ntyA02'
 '3aj4B00' '2dhiA00' '2d9vA01' '1i2hA00' '4f7uP00' '4f77e00' '1wvhA00'
 '2dkqA01' '2oqbA00' '1u5dD00' '1u5fA01' '3so6A00' '5c6rA00' '2da0A00'
 '1mai

In [20]:
DATASET.query("atom_name == 'CA' and domain =='PH'").columns

Index(['atom_number', 'atom_name', 'residue_name', 'chain_id',
       'residue_number', 'x_coord', 'y_coord', 'z_coord', 'occupancy',
       'b_factor', 'sec_struc', 'sec_struc_full', 'prot_block',
       'sasa_rel_dssp', 'ASA_res_freesasa_florian', 'RSA_freesasa_florian',
       'ASA_total_freesasa', 'ASA_mainchain_freesasa',
       'ASA_sidechain_freesasa', 'RSA_sidechain_freesasa',
       'RSA_total_freesasa_tien', 'RSA_sidechain_freesasa_tien',
       'sec_struc_segment', 'pdb', 'domain', 'cathpdb', 'uniprot_acc',
       'data_type', 'Experimental Method', 'convhull_vertex',
       'co_insertable_neighbors', 'density', 'is_co_insertable',
       'is_hydrophobic_protrusion', 'neighboursID', 'protrusion', 'LDCI',
       'S35', 'S60', 'S95', 'S100', 'S100Count', 'resolution', 'uniprot_id',
       'origin', 'residue_index', 'alignment_position', 'prositeName',
       'prositeID', 'ali_range', 'location', 'CR:prositeID', 'taxon',
       'CR:prositeName', 'uniref50', 'uniref90', 'uniref1

In [21]:
t = DATASET.query("atom_name == 'CA' and domain =='PH' and data_type == 'cathpdb'")[["ASA_res_freesasa_florian","RSA_freesasa_florian","ASA_total_freesasa","ASA_mainchain_freesasa","ASA_sidechain_freesasa","RSA_sidechain_freesasa","RSA_total_freesasa_tien","RSA_sidechain_freesasa_tien"]]